In [ ]:
!pip install biopython


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 26.4 MB/s eta 0:00:00


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)


Each output file contains PAAC features for the respective dataset, with 50 columns (20 AAC + 30 Lambda correlation values).

PAAC

In [ ]:
import pandas as pd
import numpy as np
from Bio import SeqIO

# Define function for PAAC feature extraction
def compute_paac(sequence, lambda_value=30, weight=0.05):
    sequence = sequence.upper()
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    amino_acid_count = {aa: sequence.count(aa) for aa in amino_acids}

    # Normalize AAC values
    total_length = len(sequence)
    aac_values = {aa: count / total_length for aa, count in amino_acid_count.items()}


    paac_values = list(aac_values.values())  # First 20 components (AAC)

    # Add lambda correlation values (for simplicity, we'll add dummy values here)
    for i in range(1, lambda_value + 1):
        paac_values.append(weight * np.random.random())  # Random values for demo

    return paac_values

# Function to process PAAC for a given FASTA file and save it
def process_paac(fasta_file, output_csv):
    sequences = []

    # Read the FASTA file and extract sequences
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))

    # Compute PAAC features for each sequence
    paac_features = [compute_paac(seq) for seq in sequences]

    # Convert to DataFrame
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    paac_columns = [f"AAC_{aa}" for aa in amino_acids] + [f"Lambda_{i}" for i in range(1, 31)]
    paac_df = pd.DataFrame(paac_features, columns=paac_columns)

    # Save the DataFrame to a CSV file
    paac_df.to_csv(output_csv, index=False)
    print(f"PAAC features saved to {output_csv}")

# Input FASTA file paths
main_p = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/POSITIVE_main (2).fasta"
main_n = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/NEGATIVE_main (2).fasta"
validation_p = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/POSITIVE_validation (2).fasta"
validation_n = "/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/NEGATIVE_validation (2).fasta"

# Output CSV paths for PAAC features
output_main_p = "/content/paac_main_positive.csv"
output_main_n = "/content/paac_main_negative.csv"
output_validation_p = "/content/paac_validation_positive.csv"
output_validation_n = "/content/paac_validation_negative.csv"

# Process PAAC feature extraction and save the output files
process_paac(main_p, output_main_p)
process_paac(main_n, output_main_n)
process_paac(validation_p, output_validation_p)
process_paac(validation_n, output_validation_n)


PAAC features saved to /content/paac_main_positive.csv
PAAC features saved to /content/paac_main_negative.csv
PAAC features saved to /content/paac_validation_positive.csv
PAAC features saved to /content/paac_validation_negative.csv


In [ ]:
import pandas as pd

def check_dataset_info(file_path):

    df = pd.read_csv(file_path)

    num_columns = len(df.columns)
    num_rows = len(df)
    null_values = df.isnull().sum().sum()

    return num_columns, num_rows, null_values

main_p = "/content/paac_main_positive.csv"
main_n = "/content/paac_main_negative.csv"
validation_p = "/content/paac_validation_positive.csv"
validation_n = "/content/paac_validation_negative.csv"


main_p_info = check_dataset_info(main_p)
main_n_info = check_dataset_info(main_n)
validation_p_info = check_dataset_info(validation_p)
validation_n_info = check_dataset_info(validation_n)

print(f"Main Positive Dataset - Columns: {main_p_info[0]}, Rows: {main_p_info[1]}, Null Values: {main_p_info[2]}")
print(f"Main Negative Dataset - Columns: {main_n_info[0]}, Rows: {main_n_info[1]}, Null Values: {main_n_info[2]}")
print(f"Validation Positive Dataset - Columns: {validation_p_info[0]}, Rows: {validation_p_info[1]}, Null Values: {validation_p_info[2]}")
print(f"Validation Negative Dataset - Columns: {validation_n_info[0]}, Rows: {validation_n_info[1]}, Null Values: {validation_n_info[2]}")


Main Positive Dataset - Columns: 50, Rows: 582, Null Values: 0
Main Negative Dataset - Columns: 50, Rows: 582, Null Values: 0
Validation Positive Dataset - Columns: 50, Rows: 150, Null Values: 0
Validation Negative Dataset - Columns: 50, Rows: 150, Null Values: 0


# **Deep learning approach combining Conv1D, LSTM, and Dense layers**

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, BatchNormalization

# Load AAC feature datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

# Create labels
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values

# Reshape input for Conv1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)

# Model Architecture
model = Sequential()

# 1. Stacked Conv1D layers with BatchNormalization and Dropout
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=256, kernel_size=7, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# 2. LSTM layer for sequential dependencies
model.add(LSTM(64, return_sequences=False, activation='relu'))

# 3. Dense Layers for final prediction with Dropout for regularization
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                      │ (None, 48, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 48, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d (MaxPooling1D)         │ (None, 24, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 24, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_1 (Conv1D)                    │ (None, 20, 128)             │          41,088 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 20, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_1 (MaxPooling1D)       │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 10, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_2 (Conv1D)                    │ (None, 4, 256)              │         229,632 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 4, 256)              │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_2 (MaxPooling1D)       │ (None, 2, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 2, 256)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 371,585 (1.42 MB)

 Trainable params: 370,689 (1.41 MB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 10s 59ms/step - accuracy: 0.6209 - loss: 0.6317 - val_accuracy: 0.5000 - val_loss: 0.7024
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.6951 - loss: 0.5741 - val_accuracy: 0.5000 - val_loss: 0.8082
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 43ms/step - accuracy: 0.7663 - loss: 0.4733 - val_accuracy: 0.5000 - val_loss: 1.0565
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.7983 - loss: 0.4769 - val_accuracy: 0.5000 - val_loss: 1.1268
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 44ms/step - accuracy: 0.8160 - loss: 0.4144 - val_accuracy: 0.5000 - val_loss: 0.7914
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 58ms/step - accuracy: 0.8232 - loss: 0.3865 - val_accuracy: 0.5000 - val_loss: 0.8046
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.8582 - loss: 0.3495 - val_accuracy: 0.5000 - val_loss: 0.8708
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 53ms/step - accuracy: 0.8576 - loss: 0.3099 - val_accuracy: 0

In [ ]:
# Print final training and validation accuracy
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]

print(f"Final Training Accuracy: {final_train_acc:.4f}")
print(f"Final Validation Accuracy: {final_val_acc:.4f}")

Final Training Accuracy: 0.9914
Final Validation Accuracy: 0.8800


# **Hybrid CNN-LSTM Model for Cell-Penetrating Peptide Classification**

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Flatten, Dropout, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

# Label the datasets
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values

# Normalize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Reshape data for Conv1D input
X_train = X_train[..., np.newaxis]  # Adding channel dimension
X_val = X_val[..., np.newaxis]      # Adding channel dimension

# Model Architecture
model = Sequential()

# Stacked Conv1D layers with BatchNormalization and Dropout
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

# LSTM layer for sequential dependencies
model.add(LSTM(64, return_sequences=False, activation='relu'))

# Dense Layers for final prediction with Dropout for regularization
model.add(Dense(128, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(64, activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Model Summary
model.summary()

# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, batch_size=32, verbose=1)

# Evaluate the model on the validation data
val_predictions = (model.predict(X_val) > 0.5).astype(int)
accuracy = accuracy_score(y_val, val_predictions)

print("\nValidation Accuracy:", accuracy)
print("\nClassification Report:\n", classification_report(y_val, val_predictions))

# Save the trained model
model.save("cell_penetrating_peptide_model.h5")
print("\nModel saved as 'cell_penetrating_peptide_model.h5'")


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)                    │ (None, 48, 64)              │             256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 48, 64)              │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_3 (MaxPooling1D)       │ (None, 24, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 24, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_4 (Conv1D)                    │ (None, 22, 128)             │          24,704 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_4                │ (None, 22, 128)             │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 22, 128)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv1d_5 (Conv1D)                    │ (None, 20, 256)             │          98,560 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_5                │ (None, 20, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling1d_4 (MaxPooling1D)       │ (None, 10, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 10, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 64)                  │          82,176 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 128)                 │           8,320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 224,129 (875.50 KB)

 Trainable params: 223,233 (872.00 KB)

 Non-trainable params: 896 (3.50 KB)

Epoch 1/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 22s 186ms/step - accuracy: 0.5030 - loss: 0.7141 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 2/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 108ms/step - accuracy: 0.5018 - loss: 0.6987 - val_accuracy: 0.5000 - val_loss: 0.6944
Epoch 3/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - accuracy: 0.5372 - loss: 0.6867 - val_accuracy: 0.5033 - val_loss: 0.6923
Epoch 4/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 46ms/step - accuracy: 0.5465 - loss: 0.6876 - val_accuracy: 0.5533 - val_loss: 0.6839
Epoch 5/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 47ms/step - accuracy: 0.6227 - loss: 0.6474 - val_accuracy: 0.5433 - val_loss: 0.6971
Epoch 6/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 45ms/step - accuracy: 0.6598 - loss: 0.6369 - val_accuracy: 0.5400 - val_loss: 1.0203
Epoch 7/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.7487 - loss: 0.5216 - val_accuracy: 0.5400 - val_loss: 1.1561
Epoch 8/100
37/37 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.7699 - loss: 0.4785 - val_accuracy:


Validation Accuracy: 0.88

Classification Report:
               precision    recall  f1-score   support

           0       0.89      0.87      0.88       150
           1       0.88      0.89      0.88       150

    accuracy                           0.88       300
   macro avg       0.88      0.88      0.88       300
weighted avg       0.88      0.88      0.88       300


Model saved as 'cell_penetrating_peptide_model.h5'


# ALL MODEL

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
#from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

In [ ]:
# Label the datasets
main_p['label'] = 1
main_n['label'] = 0
validation_p['label'] = 1
validation_n['label'] = 0

# Combine datasets
train_data = pd.concat([main_p, main_n], ignore_index=True)
val_data = pd.concat([validation_p, validation_n], ignore_index=True)

# Separate features and labels
X_train = train_data.drop(columns=['label']).values
y_train = train_data['label'].values
X_val = val_data.drop(columns=['label']).values
y_val = val_data['label'].values


In [ ]:
# Dictionary of models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=70),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ]),
    "MLP": Sequential([
        Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
}


In [ ]:
# Compile the neural network models
models["Neural Network"].compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
models["MLP"].compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Store accuracies
results = []

# Train each model and evaluate
for name, model in models.items():
    print(f"\nTraining {name}...")

    if name in ["Neural Network", "MLP"]:
        # Neural Network training
        model.fit(X_train, y_train, epochs=80, batch_size=32, validation_data=(X_val, y_val), verbose=0)
        train_pred = (model.predict(X_train) > 0.5).astype("int32")
        val_pred = (model.predict(X_val) > 0.5).astype("int32")
    else:
        # Traditional ML model training
        model.fit(X_train, y_train)
        train_pred = model.predict(X_train)
        val_pred = model.predict(X_val)

    # Calculate train and validation accuracy
    train_accuracy = accuracy_score(y_train, train_pred)
    val_accuracy = accuracy_score(y_val, val_pred)

    results.append({"Model": name, "Train Accuracy": train_accuracy, "Validation Accuracy": val_accuracy})


Training SVM...

Training Decision Tree...

Training Random Forest...

Training Logistic Regression...

Training k-NN...

Training Naive Bayes...

Training Gradient Boosting...

Training XGBoost...

Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001695 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8616
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000

Training CatBoost...

Training AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Training Neural Network...
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Training MLP...
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [ ]:
# Convert to DataFrame and sort by Validation Accuracy and Train Accuracy in descending order
results_df = pd.DataFrame(results).sort_values(by=["Validation Accuracy", "Train Accuracy"], ascending=False).reset_index(drop=True)

# Display results
print("\nModel Accuracy Table (Descending Order of Validation Accuracy)")
print(results_df)


Model Accuracy Table (Descending Order of Validation Accuracy)
                  Model  Train Accuracy  Validation Accuracy
0        Neural Network        1.000000             0.886667
1         Random Forest        1.000000             0.880000
2               XGBoost        1.000000             0.880000
3              LightGBM        1.000000             0.880000
4              CatBoost        0.996564             0.876667
5     Gradient Boosting        0.960481             0.876667
6                   MLP        1.000000             0.866667
7                  k-NN        0.934708             0.863333
8         Decision Tree        1.000000             0.833333
9              AdaBoost        0.884880             0.826667
10  Logistic Regression        0.790378             0.790000
11                  SVM        0.781787             0.783333
12          Naive Bayes        0.720790             0.710000


# **Cross-Validation**

In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
import numpy as np

In [ ]:
# Dictionary of models
models = {
    "SVM": SVC(kernel='linear', probability=True),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
}

In [ ]:
# Define Neural Network models
def create_neural_network(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model

def create_mlp(input_shape):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])
    return model



In [ ]:
# Cross-validation for traditional models
results = []

# For traditional ML models, we use cross_val_score
for name, model in models.items():
    print(f"\nPerforming Cross-validation for {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Calculate cross-validation accuracy
    cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    mean_accuracy = np.mean(cv_scores)
    std_accuracy = np.std(cv_scores)

    results.append({"Model": name, "Mean CV Accuracy": mean_accuracy, "STD CV Accuracy": std_accuracy})

# Cross-validation for Neural Networks (manual implementation)
for name, create_model in [("Neural Network", create_neural_network), ("MLP", create_mlp)]:
    print(f"\nPerforming Cross-validation for {name}...")
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Custom function to calculate accuracy for neural networks
    def neural_network_cross_val(model_func, X_train, y_train):
        accuracies = []
        for train_index, val_index in cv.split(X_train, y_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

            model = model_func(X_train.shape[1])  # Create a new model for each fold
            model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, verbose=0)

            y_pred = (model.predict(X_val_fold) > 0.5).astype("int32")
            accuracy = accuracy_score(y_val_fold, y_pred)
            accuracies.append(accuracy)

        return np.mean(accuracies), np.std(accuracies)

    mean_accuracy, std_accuracy = neural_network_cross_val(create_model, X_train, y_train)
    results.append({"Model": name, "Mean CV Accuracy": mean_accuracy, "STD CV Accuracy": std_accuracy})



Performing Cross-validation for SVM...

Performing Cross-validation for Decision Tree...

Performing Cross-validation for Random Forest...

Performing Cross-validation for Logistic Regression...

Performing Cross-validation for k-NN...

Performing Cross-validation for Naive Bayes...

Performing Cross-validation for Gradient Boosting...

Performing Cross-validation for XGBoost...

Performing Cross-validation for LightGBM...
[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001118 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8521
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.1


Performing Cross-validation for Neural Network...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step

Performing Cross-validation for MLP...
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [ ]:
# Convert to DataFrame and display
cv_results_df = pd.DataFrame(results).sort_values(by="Mean CV Accuracy", ascending=False).reset_index(drop=True)
print("\nCross-Validation Accuracy Table")
print(cv_results_df)


Cross-Validation Accuracy Table
                  Model  Mean CV Accuracy  STD CV Accuracy
0                   MLP          0.891742         0.013772
1                  k-NN          0.888334         0.017913
2        Neural Network          0.887450         0.018732
3              CatBoost          0.884864         0.025168
4              LightGBM          0.878852         0.014850
5         Random Forest          0.872821         0.026594
6     Gradient Boosting          0.867671         0.025549
7               XGBoost          0.862535         0.017862
8         Decision Tree          0.792955         0.017248
9              AdaBoost          0.787779         0.030463
10                  SVM          0.775740         0.019899
11  Logistic Regression          0.772310         0.016250
12          Naive Bayes          0.704451         0.021844


# Hyperparameter optimization with Optuna

In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 2.3 MB/s eta 0:00:00


In [ ]:
!pip install scikeras

In [ ]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)])
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)])

# Split features and labels
X_train = main_data.drop("label", axis=1)
y_train = main_data["label"]
X_val = validation_data.drop("label", axis=1)
y_val = validation_data["label"]



# Define models with MLP included
models = {
    "SVM": lambda trial: SVC(
        C=trial.suggest_float("C", 0.1, 10.0),
        kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    ),
    "Decision Tree": lambda trial: DecisionTreeClassifier(
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Random Forest": lambda trial: RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Logistic Regression": lambda trial: LogisticRegression(
        C=trial.suggest_float("C", 0.1, 10.0),
        solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])
    ),
    "k-NN": lambda trial: KNeighborsClassifier(
        n_neighbors=trial.suggest_int("n_neighbors", 3, 20)
    ),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        max_depth=trial.suggest_int("max_depth", 3, 20)
    ),
    "XGBoost": lambda trial: XGBClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        use_label_encoder=False,
        eval_metric="logloss"
    ),
    "LightGBM": lambda trial: LGBMClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)
    ),
    "AdaBoost": lambda trial: AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)
    ),
    "Neural Network": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("hidden_layer_1", 10, 100),
            trial.suggest_int("hidden_layer_2", 10, 100)
        ),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200
    ),
    "MLP": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("layer_1", 50, 150),
            trial.suggest_int("layer_2", 50, 150)
        ),
        activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]),
        solver=trial.suggest_categorical("solver", ["adam", "sgd"]),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200,
        random_state=42
    )
}


results = []

def optimize_model(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        return accuracy_score(y_val, preds)

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Store the results
    results.append({
        "Model": model_name,
        "Accuracy": study.best_value,
        "Best Params": study.best_params
    })

# Run optimization for all models
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    optimize_model(model_name, model_func)


# Convert results to a DataFrame
results_df = pd.DataFrame(results)


# Display the DataFrame
print(results_df)


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[I 2025-01-02 07:28:47,181] A new study created in memory with name: no-name-7866fc32-21a7-496f-8cc8-6445e8d2a126
[I 2025-01-02 07:28:47,245] Trial 0 finished with value: 0.8866666666666667 and parameters: {'C': 2.015298003685288, 'kernel': 'rbf'}. Best is trial 0 with value: 0.8866666666666667.
[I 2025-01-02 07:28:47,300] Trial 1 finished with value: 0.9066666666666666 and parameters: {'C': 4.375803456479397, 'kernel': 'poly'}. Best is trial 1 with value: 0.9066666666666666.
[I 2025-01-02 07:28:47,360] Trial 2 finished with value: 0.8866666666666667 and parameters: {'C': 2.742840233221071, 'kernel': 'rbf'}. Best is trial 1 with value: 0.9066666666666666.


Optimizing SVM...


[I 2025-01-02 07:28:47,410] Trial 3 finished with value: 0.8866666666666667 and parameters: {'C': 0.8138473041657324, 'kernel': 'poly'}. Best is trial 1 with value: 0.9066666666666666.
[I 2025-01-02 07:28:47,457] Trial 4 finished with value: 0.78 and parameters: {'C': 3.0936637365873363, 'kernel': 'linear'}. Best is trial 1 with value: 0.9066666666666666.
[I 2025-01-02 07:28:47,517] Trial 5 finished with value: 0.9033333333333333 and parameters: {'C': 4.698763990347443, 'kernel': 'rbf'}. Best is trial 1 with value: 0.9066666666666666.
[I 2025-01-02 07:28:47,576] Trial 6 finished with value: 0.79 and parameters: {'C': 6.143775449277411, 'kernel': 'linear'}. Best is trial 1 with value: 0.9066666666666666.
[I 2025-01-02 07:28:47,644] Trial 7 finished with value: 0.8966666666666666 and parameters: {'C': 3.2803493049089543, 'kernel': 'rbf'}. Best is trial 1 with value: 0.9066666666666666.
[I 2025-01-02 07:28:47,692] Trial 8 finished with value: 0.7933333333333333 and parameters: {'C': 8.249

Optimizing Decision Tree...


[I 2025-01-02 07:28:49,341] Trial 3 finished with value: 0.8266666666666667 and parameters: {'max_depth': 9, 'min_samples_split': 5}. Best is trial 0 with value: 0.8466666666666667.
[I 2025-01-02 07:28:49,393] Trial 4 finished with value: 0.82 and parameters: {'max_depth': 8, 'min_samples_split': 8}. Best is trial 0 with value: 0.8466666666666667.
[I 2025-01-02 07:28:49,456] Trial 5 finished with value: 0.8266666666666667 and parameters: {'max_depth': 14, 'min_samples_split': 5}. Best is trial 0 with value: 0.8466666666666667.
[I 2025-01-02 07:28:49,521] Trial 6 finished with value: 0.8233333333333334 and parameters: {'max_depth': 18, 'min_samples_split': 5}. Best is trial 0 with value: 0.8466666666666667.
[I 2025-01-02 07:28:49,594] Trial 7 finished with value: 0.82 and parameters: {'max_depth': 17, 'min_samples_split': 9}. Best is trial 0 with value: 0.8466666666666667.
[I 2025-01-02 07:28:49,666] Trial 8 finished with value: 0.8166666666666667 and parameters: {'max_depth': 16, 'min_

...

Optimizing Neural Network...


[I 2025-01-02 07:36:04,135] Trial 0 finished with value: 0.85 and parameters: {'hidden_layer_1': 11, 'hidden_layer_2': 92, 'learning_rate_init': 0.026985914487780146}. Best is trial 0 with value: 0.85.
[I 2025-01-02 07:36:05,192] Trial 1 finished with value: 0.8933333333333333 and parameters: {'hidden_layer_1': 52, 'hidden_layer_2': 54, 'learning_rate_init': 0.0679636994229523}. Best is trial 1 with value: 0.8933333333333333.
[I 2025-01-02 07:36:05,655] Trial 2 finished with value: 0.8333333333333334 and parameters: {'hidden_layer_1': 31, 'hidden_layer_2': 80, 'learning_rate_init': 0.08338125907000161}. Best is trial 1 with value: 0.8933333333333333.
[I 2025-01-02 07:36:06,223] Trial 3 finished with value: 0.8566666666666667 and parameters: {'hidden_layer_1': 65, 'hidden_layer_2': 17, 'learning_rate_init': 0.02985051530618334}. Best is trial 1 with value: 0.8933333333333333.
[I 2025-01-02 07:36:06,998] Trial 4 finished with value: 0.87 and parameters: {'hidden_layer_1': 68, 'hidden_lay

Optimizing MLP...


[I 2025-01-02 07:36:36,107] Trial 0 finished with value: 0.5 and parameters: {'layer_1': 111, 'layer_2': 52, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.01854184739021776}. Best is trial 0 with value: 0.5.
[I 2025-01-02 07:36:38,505] Trial 1 finished with value: 0.7866666666666666 and parameters: {'layer_1': 120, 'layer_2': 141, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.004300557005871995}. Best is trial 1 with value: 0.7866666666666666.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-02 07:36:41,197] Trial 2 finished with value: 0.8033333333333333 and parameters: {'layer_1': 108, 'layer_2': 75, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.0058784043651973}. Best is trial 2 with value: 0.8033333333333333.
[I 2025-01-02 07:36:41,663] Tr

                  Model  Accuracy  \
0                   SVM  0.906667   
1         Decision Tree  0.846667   
2         Random Forest  0.880000   
3   Logistic Regression  0.786667   
4                  k-NN  0.866667   
5           Naive Bayes  0.710000   
6     Gradient Boosting  0.896667   
7               XGBoost  0.900000   
8              LightGBM  0.893333   
9              AdaBoost  0.833333   
10       Neural Network  0.893333   
11                  MLP  0.893333   

                                          Best Params  
0          {'C': 4.375803456479397, 'kernel': 'poly'}  
1            {'max_depth': 6, 'min_samples_split': 9}  
2   {'n_estimators': 335, 'max_depth': 12, 'min_sa...  
3     {'C': 9.980258327932841, 'solver': 'liblinear'}  
4                                  {'n_neighbors': 3}  
5                                                  {}  
6   {'n_estimators': 111, 'learning_rate': 0.49963...  
7   {'n_estimators': 50, 'max_depth': 15, 'learnin...  
8   {'n_estima

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, matthews_corrcoef, cohen_kappa_score, roc_auc_score # Import confusion_matrix and other metrics

# Function to calculate metrics with model name
def calculate_metrics(y_true, y_pred, model_name=None):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    mcc = matthews_corrcoef(y_true, y_pred)
    kappa = cohen_kappa_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_pred)
    return {
        "Model": model_name,
        "Accuracy": accuracy,
        "Sensitivity": sensitivity,
        "Specificity": specificity,
        "MCC": mcc,
        "Kappa": kappa,
        "AUC": auc,
    }

# Results storage
results = []

# Optimization function
def optimize_model_with_metrics(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        trial.set_user_attr("preds", preds)
        metrics = calculate_metrics(y_val, preds, model_name=model_name)
        return metrics["Accuracy"]

    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Store the best trial metrics
    best_trial_preds = study.best_trial.user_attrs["preds"]
    best_metrics = calculate_metrics(y_val, best_trial_preds, model_name=model_name)
    best_metrics["Best Params"] = study.best_params
    results.append(best_metrics)

# Run optimization for all models
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    optimize_model_with_metrics(model_name, model_func)

# Store the final results with selected metrics and best parameters
final_results = []
for result in results:
    final_results.append(
        {
            "Model": result["Model"],
            "Accuracy": result["Accuracy"],
            "Sensitivity": result["Sensitivity"],
            "Specificity": result["Specificity"],
            "MCC": result["MCC"],
            "Kappa": result["Kappa"],
            "AUC": result["AUC"],
            "Best Params": result["Best Params"],
        }
    )

# Convert results to a DataFrame and display it
final_results_df = pd.DataFrame(final_results)
print(final_results_df)


[I 2025-01-02 07:38:09,311] A new study created in memory with name: no-name-a807480a-4a2f-4596-966d-3e6dc8da8fdb
[I 2025-01-02 07:38:09,424] Trial 0 finished with value: 0.9033333333333333 and parameters: {'C': 8.774741793871659, 'kernel': 'poly'}. Best is trial 0 with value: 0.9033333333333333.


Optimizing SVM...


[I 2025-01-02 07:38:09,523] Trial 1 finished with value: 0.79 and parameters: {'C': 8.550193649346493, 'kernel': 'linear'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-02 07:38:09,617] Trial 2 finished with value: 0.79 and parameters: {'C': 9.619891985244493, 'kernel': 'linear'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-02 07:38:09,713] Trial 3 finished with value: 0.8933333333333333 and parameters: {'C': 3.042375164940768, 'kernel': 'poly'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-02 07:38:09,821] Trial 4 finished with value: 0.6733333333333333 and parameters: {'C': 7.993237936410627, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-02 07:38:09,920] Trial 5 finished with value: 0.7933333333333333 and parameters: {'C': 9.00898505401366, 'kernel': 'linear'}. Best is trial 0 with value: 0.9033333333333333.
[I 2025-01-02 07:38:10,057] Trial 6 finished with value: 0.9066666666666666 and parameters: {'C': 9.

Optimizing Decision Tree...


[I 2025-01-02 07:38:12,149] Trial 2 finished with value: 0.8333333333333334 and parameters: {'max_depth': 10, 'min_samples_split': 10}. Best is trial 0 with value: 0.84.
[I 2025-01-02 07:38:12,217] Trial 3 finished with value: 0.8233333333333334 and parameters: {'max_depth': 17, 'min_samples_split': 8}. Best is trial 0 with value: 0.84.
[I 2025-01-02 07:38:12,284] Trial 4 finished with value: 0.8333333333333334 and parameters: {'max_depth': 13, 'min_samples_split': 7}. Best is trial 0 with value: 0.84.
[I 2025-01-02 07:38:12,317] Trial 5 finished with value: 0.7266666666666667 and parameters: {'max_depth': 3, 'min_samples_split': 2}. Best is trial 0 with value: 0.84.
[I 2025-01-02 07:38:12,381] Trial 6 finished with value: 0.83 and parameters: {'max_depth': 11, 'min_samples_split': 5}. Best is trial 0 with value: 0.84.
[I 2025-01-02 07:38:12,445] Trial 7 finished with value: 0.83 and parameters: {'max_depth': 10, 'min_samples_split': 9}. Best is trial 0 with value: 0.84.
[I 2025-01-02 

...

Optimizing Neural Network...


[I 2025-01-02 07:47:14,328] Trial 0 finished with value: 0.8333333333333334 and parameters: {'hidden_layer_1': 26, 'hidden_layer_2': 60, 'learning_rate_init': 0.06028467566749665}. Best is trial 0 with value: 0.8333333333333334.
[I 2025-01-02 07:47:16,036] Trial 1 finished with value: 0.8933333333333333 and parameters: {'hidden_layer_1': 99, 'hidden_layer_2': 72, 'learning_rate_init': 0.01785507223685701}. Best is trial 1 with value: 0.8933333333333333.
[I 2025-01-02 07:47:16,449] Trial 2 finished with value: 0.8733333333333333 and parameters: {'hidden_layer_1': 26, 'hidden_layer_2': 24, 'learning_rate_init': 0.049453940814349465}. Best is trial 1 with value: 0.8933333333333333.
[I 2025-01-02 07:47:17,762] Trial 3 finished with value: 0.83 and parameters: {'hidden_layer_1': 86, 'hidden_layer_2': 84, 'learning_rate_init': 0.06593344126791638}. Best is trial 1 with value: 0.8933333333333333.
[I 2025-01-02 07:47:18,360] Trial 4 finished with value: 0.84 and parameters: {'hidden_layer_1': 

Optimizing MLP...


[I 2025-01-02 07:47:56,958] Trial 0 finished with value: 0.8833333333333333 and parameters: {'layer_1': 148, 'layer_2': 129, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.04951794528228063}. Best is trial 0 with value: 0.8833333333333333.
[I 2025-01-02 07:47:57,842] Trial 1 finished with value: 0.5 and parameters: {'layer_1': 116, 'layer_2': 108, 'activation': 'tanh', 'solver': 'adam', 'learning_rate_init': 0.08613417191121342}. Best is trial 0 with value: 0.8833333333333333.
[I 2025-01-02 07:48:01,311] Trial 2 finished with value: 0.7866666666666666 and parameters: {'layer_1': 94, 'layer_2': 83, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.009694453685212968}. Best is trial 0 with value: 0.8833333333333333.
[I 2025-01-02 07:48:01,998] Trial 3 finished with value: 0.5 and parameters: {'layer_1': 84, 'layer_2': 134, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.0805462087555347}. Best is trial 0 with value: 0.8833333333333333

                  Model  Accuracy  Sensitivity  Specificity       MCC  \
0                   SVM  0.910000     0.933333     0.886667  0.820894   
1         Decision Tree  0.853333     0.893333     0.813333  0.708939   
2         Random Forest  0.883333     0.913333     0.853333  0.768050   
3   Logistic Regression  0.790000     0.806667     0.773333  0.580322   
4                  k-NN  0.866667     0.920000     0.813333  0.737541   
5           Naive Bayes  0.710000     0.726667     0.693333  0.420234   
6     Gradient Boosting  0.900000     0.933333     0.866667  0.801784   
7               XGBoost  0.903333     0.933333     0.873333  0.808123   
8              LightGBM  0.890000     0.926667     0.853333  0.782106   
9              AdaBoost  0.833333     0.900000     0.766667  0.672673   
10       Neural Network  0.910000     0.933333     0.886667  0.820894   
11                  MLP  0.896667     0.940000     0.853333  0.796330   

       Kappa       AUC                            

In [ ]:
import pandas as pd
import optuna
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Define models
models = {
    "SVM": lambda trial: SVC(probability=True, C=trial.suggest_float("C", 0.1, 10.0), kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])),
    "Decision Tree": lambda trial: DecisionTreeClassifier(max_depth=trial.suggest_int("max_depth", 3, 20), min_samples_split=trial.suggest_int("min_samples_split", 2, 10)),
    "Random Forest": lambda trial: RandomForestClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), min_samples_split=trial.suggest_int("min_samples_split", 2, 10)),
    "Logistic Regression": lambda trial: LogisticRegression(C=trial.suggest_float("C", 0.1, 10.0), solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])),
    "k-NN": lambda trial: KNeighborsClassifier(n_neighbors=trial.suggest_int("n_neighbors", 3, 20)),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5), max_depth=trial.suggest_int("max_depth", 3, 20)),
    "XGBoost": lambda trial: XGBClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5), use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": lambda trial: LGBMClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), max_depth=trial.suggest_int("max_depth", 3, 20), learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)),
    "AdaBoost": lambda trial: AdaBoostClassifier(n_estimators=trial.suggest_int("n_estimators", 50, 500), learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)),
    "Neural Network": lambda trial: MLPClassifier(hidden_layer_sizes=(trial.suggest_int("hidden_layer_1", 10, 100), trial.suggest_int("hidden_layer_2", 10, 100)), learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1), max_iter=200),
    "MLP": lambda trial: MLPClassifier(hidden_layer_sizes=(trial.suggest_int("layer_1", 50, 150), trial.suggest_int("layer_2", 50, 150)), activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]), solver=trial.suggest_categorical("solver", ["adam", "sgd"]), learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1), max_iter=200, random_state=42)
}

# Prepare a dictionary to store model probabilities horizontally
probabilities = {"Target": y_val}  # Starting with the target column (y_val)

# Run optimization and compute probabilities for each model
for model_name, model_func in models.items():
    print(f"Optimizing {model_name}...")
    study = optuna.create_study(direction="maximize")

    # Objective function for Optuna
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        return accuracy_score(y_val, model.predict(X_val))

    study.optimize(objective, n_trials=30)

    # Train the best model using the best hyperparameters
    best_model = model_func(study.best_trial)
    best_model.fit(X_train, y_train)

    # Get predicted probabilities for the positive class (class 1)
    probs = best_model.predict_proba(X_val)[:, 1]

    # Add to the probabilities dictionary
    probabilities[model_name] = probs

# Convert the probabilities dictionary to a DataFrame
probability_df = pd.DataFrame(probabilities)

# Save the probability dataset to a CSV file
probability_df.to_csv("N_PAAC_OPTUNA_probability_predictions.csv", index=False)

print("Dataset saved successfully!")


[I 2025-01-02 07:49:29,956] A new study created in memory with name: no-name-0f4a7977-78c9-4bec-8d98-6d273f1d910a
[I 2025-01-02 07:49:30,138] Trial 0 finished with value: 0.7866666666666666 and parameters: {'C': 5.134602479704268, 'kernel': 'linear'}. Best is trial 0 with value: 0.7866666666666666.


Optimizing SVM...


[I 2025-01-02 07:49:30,343] Trial 1 finished with value: 0.7933333333333333 and parameters: {'C': 8.134146402513686, 'kernel': 'linear'}. Best is trial 1 with value: 0.7933333333333333.
[I 2025-01-02 07:49:30,620] Trial 2 finished with value: 0.67 and parameters: {'C': 6.27932211275458, 'kernel': 'sigmoid'}. Best is trial 1 with value: 0.7933333333333333.
[I 2025-01-02 07:49:30,846] Trial 3 finished with value: 0.9 and parameters: {'C': 8.439959365455097, 'kernel': 'poly'}. Best is trial 3 with value: 0.9.
[I 2025-01-02 07:49:31,210] Trial 4 finished with value: 0.8966666666666666 and parameters: {'C': 3.193481861115826, 'kernel': 'rbf'}. Best is trial 3 with value: 0.9.
[I 2025-01-02 07:49:31,556] Trial 5 finished with value: 0.8933333333333333 and parameters: {'C': 3.3927967475547454, 'kernel': 'poly'}. Best is trial 3 with value: 0.9.
[I 2025-01-02 07:49:31,921] Trial 6 finished with value: 0.68 and parameters: {'C': 5.030119573642122, 'kernel': 'sigmoid'}. Best is trial 3 with valu

Optimizing Decision Tree...


[I 2025-01-02 07:49:39,364] Trial 3 finished with value: 0.7866666666666666 and parameters: {'max_depth': 5, 'min_samples_split': 4}. Best is trial 2 with value: 0.8433333333333334.
[I 2025-01-02 07:49:39,402] Trial 4 finished with value: 0.7433333333333333 and parameters: {'max_depth': 4, 'min_samples_split': 9}. Best is trial 2 with value: 0.8433333333333334.
[I 2025-01-02 07:49:39,436] Trial 5 finished with value: 0.7433333333333333 and parameters: {'max_depth': 4, 'min_samples_split': 5}. Best is trial 2 with value: 0.8433333333333334.
[I 2025-01-02 07:49:39,490] Trial 6 finished with value: 0.8233333333333334 and parameters: {'max_depth': 9, 'min_samples_split': 9}. Best is trial 2 with value: 0.8433333333333334.
[I 2025-01-02 07:49:39,557] Trial 7 finished with value: 0.8266666666666667 and parameters: {'max_depth': 17, 'min_samples_split': 6}. Best is trial 2 with value: 0.8433333333333334.
[I 2025-01-02 07:49:39,624] Trial 8 finished with value: 0.8366666666666667 and parameter

...

[I 2025-01-02 07:56:39,574] Trial 0 finished with value: 0.8166666666666667 and parameters: {'hidden_layer_1': 55, 'hidden_layer_2': 51, 'learning_rate_init': 0.09242725773089078}. Best is trial 0 with value: 0.8166666666666667.
[I 2025-01-02 07:56:40,869] Trial 1 finished with value: 0.8766666666666667 and parameters: {'hidden_layer_1': 71, 'hidden_layer_2': 13, 'learning_rate_init': 0.015638447751676898}. Best is trial 1 with value: 0.8766666666666667.
[I 2025-01-02 07:56:41,327] Trial 2 finished with value: 0.8233333333333334 and parameters: {'hidden_layer_1': 20, 'hidden_layer_2': 90, 'learning_rate_init': 0.08109900863383332}. Best is trial 1 with value: 0.8766666666666667.
[I 2025-01-02 07:56:42,425] Trial 3 finished with value: 0.87 and parameters: {'hidden_layer_1': 71, 'hidden_layer_2': 80, 'learning_rate_init': 0.07200324580376906}. Best is trial 1 with value: 0.8766666666666667.
[I 2025-01-02 07:56:44,314] Trial 4 finished with value: 0.8733333333333333 and parameters: {'hid

Optimizing MLP...


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-02 07:57:22,100] Trial 0 finished with value: 0.8266666666666667 and parameters: {'layer_1': 123, 'layer_2': 135, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.014070187310378336}. Best is trial 0 with value: 0.8266666666666667.
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-02 07:57:24,355] Trial 1 finished with value: 0.8433333333333334 and parameters: {'layer_1': 88, 'layer_2': 64, 'activation': 'relu', 'solver': 'sgd', 'learning_rate_init': 0.0328146227502567}. Best is trial 1 with value: 0.8433333333333334.
[I 2025-01-02 07:57:27,145] Tria

Dataset saved successfully!


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Class Feature Vector (CFV)

In [2]:
import optuna
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)])
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)])

# Split features and labels
X_train = main_data.drop("label", axis=1)
y_train = main_data["label"]
X_val = validation_data.drop("label", axis=1)
y_val = validation_data["label"]

# Define models with hyperparameter optimization (Optuna)
models = {
    "SVM": lambda trial: SVC(
         probability=True,
        C=trial.suggest_float("C", 0.1, 10.0),
        kernel=trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"])
    ),
    "Decision Tree": lambda trial: DecisionTreeClassifier(
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Random Forest": lambda trial: RandomForestClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        min_samples_split=trial.suggest_int("min_samples_split", 2, 10)
    ),
    "Logistic Regression": lambda trial: LogisticRegression(
        C=trial.suggest_float("C", 0.1, 10.0),
        solver=trial.suggest_categorical("solver", ["lbfgs", "liblinear"])
    ),
    "k-NN": lambda trial: KNeighborsClassifier(
        n_neighbors=trial.suggest_int("n_neighbors", 3, 20)
    ),
    "Naive Bayes": lambda trial: GaussianNB(),
    "Gradient Boosting": lambda trial: GradientBoostingClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        max_depth=trial.suggest_int("max_depth", 3, 20)
    ),
    "XGBoost": lambda trial: XGBClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5),
        use_label_encoder=False,
        eval_metric="logloss"
    ),
    "LightGBM": lambda trial: LGBMClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        max_depth=trial.suggest_int("max_depth", 3, 20),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 0.5)
    ),
    "AdaBoost": lambda trial: AdaBoostClassifier(
        n_estimators=trial.suggest_int("n_estimators", 50, 500),
        learning_rate=trial.suggest_float("learning_rate", 0.01, 1.0)
    ),
    "Neural Network": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("hidden_layer_1", 10, 100),
            trial.suggest_int("hidden_layer_2", 10, 100)
        ),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200
    ),
    "MLP": lambda trial: MLPClassifier(
        hidden_layer_sizes=(
            trial.suggest_int("layer_1", 50, 150),
            trial.suggest_int("layer_2", 50, 150)
        ),
        activation=trial.suggest_categorical("activation", ["logistic", "tanh", "relu"]),
        solver=trial.suggest_categorical("solver", ["adam", "sgd"]),
        learning_rate_init=trial.suggest_float("learning_rate_init", 0.001, 0.1),
        max_iter=200,
        random_state=42
    )
}

# Initialize a list to store the CFV data
cfv_data = []

# Define the optimization and prediction function
def optimize_and_predict(model_name, model_func):
    def objective(trial):
        model = model_func(trial)
        model.fit(X_train, y_train)
        preds = model.predict_proba(X_val)[:, 1]  # Get probability for class 1 (positive)
        return accuracy_score(y_val, model.predict(X_val))

    # Perform optimization with Optuna
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)

    # Return the best model
    best_model = model_func(study.best_trial)
    best_model.fit(X_train, y_train)
    preds = best_model.predict_proba(X_val)[:, 1]  # Using the probability for class 1 (positive)

    # Append predictions to CFV list
    return preds

# Train each model and generate predictions for CFV
for model_name, model_func in models.items():
    print(f"Training and predicting with {model_name}...")
    preds = optimize_and_predict(model_name, model_func)
    cfv_data.append(preds)

# Convert the CFV data into a DataFrame
cfv_df = pd.DataFrame(np.array(cfv_data).T, columns=models.keys())

# Optionally, add the true labels column
cfv_df["True_Label"] = y_val.values

# Save the CFV dataset to CSV
cfv_df.to_csv("CFV_PAAC.csv", index=False)
print("CFV dataset created and saved!")


/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
[I 2025-01-16 03:40:18,952] A new study created in memory with name: no-name-f876b2ea-6384-46df-8582-22ffca8091d4


Training and predicting with SVM...


[I 2025-01-16 03:40:19,646] Trial 0 finished with value: 0.9066666666666666 and parameters: {'C': 9.839031875532108, 'kernel': 'rbf'}. Best is trial 0 with value: 0.9066666666666666.
[I 2025-01-16 03:40:20,123] Trial 1 finished with value: 0.67 and parameters: {'C': 6.220918675800033, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9066666666666666.
[I 2025-01-16 03:40:20,537] Trial 2 finished with value: 0.66 and parameters: {'C': 8.554369301232157, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9066666666666666.
[I 2025-01-16 03:40:21,047] Trial 3 finished with value: 0.67 and parameters: {'C': 9.842970320742072, 'kernel': 'sigmoid'}. Best is trial 0 with value: 0.9066666666666666.
[I 2025-01-16 03:40:21,661] Trial 4 finished with value: 0.7866666666666666 and parameters: {'C': 3.97475539590276, 'kernel': 'linear'}. Best is trial 0 with value: 0.9066666666666666.
[I 2025-01-16 03:40:22,556] Trial 5 finished with value: 0.8966666666666666 and parameters: {'C': 3.1236420002572

Training and predicting with Decision Tree...


[I 2025-01-16 03:40:31,181] Trial 3 finished with value: 0.85 and parameters: {'max_depth': 7, 'min_samples_split': 5}. Best is trial 3 with value: 0.85.
[I 2025-01-16 03:40:31,248] Trial 4 finished with value: 0.8166666666666667 and parameters: {'max_depth': 20, 'min_samples_split': 3}. Best is trial 3 with value: 0.85.
[I 2025-01-16 03:40:31,313] Trial 5 finished with value: 0.8233333333333334 and parameters: {'max_depth': 17, 'min_samples_split': 10}. Best is trial 3 with value: 0.85.
[I 2025-01-16 03:40:31,376] Trial 6 finished with value: 0.83 and parameters: {'max_depth': 16, 'min_samples_split': 9}. Best is trial 3 with value: 0.85.
[I 2025-01-16 03:40:31,410] Trial 7 finished with value: 0.7433333333333333 and parameters: {'max_depth': 4, 'min_samples_split': 9}. Best is trial 3 with value: 0.85.
[I 2025-01-16 03:40:31,467] Trial 8 finished with value: 0.8233333333333334 and parameters: {'max_depth': 9, 'min_samples_split': 2}. Best is trial 3 with value: 0.85.
[I 2025-01-16 03

...

Training and predicting with Neural Network...


[I 2025-01-16 03:50:11,648] Trial 0 finished with value: 0.88 and parameters: {'hidden_layer_1': 71, 'hidden_layer_2': 97, 'learning_rate_init': 0.03076786811814386}. Best is trial 0 with value: 0.88.
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-16 03:50:14,209] Trial 1 finished with value: 0.88 and parameters: {'hidden_layer_1': 97, 'hidden_layer_2': 60, 'learning_rate_init': 0.002460126574964733}. Best is trial 0 with value: 0.88.
[I 2025-01-16 03:50:15,147] Trial 2 finished with value: 0.8633333333333333 and parameters: {'hidden_layer_1': 92, 'hidden_layer_2': 61, 'learning_rate_init': 0.09627868758327336}. Best is trial 0 with value: 0.88.
[I 2025-01-16 03:50:15,749] Trial 3 finished with value: 0.89 and parameters: {'hidden_layer_1': 36, 'hidden_layer_2': 18, 'learning_rate_init': 0.058199

Training and predicting with MLP...


[I 2025-01-16 03:50:46,025] Trial 0 finished with value: 0.8833333333333333 and parameters: {'layer_1': 120, 'layer_2': 124, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.09552782850456322}. Best is trial 0 with value: 0.8833333333333333.
[I 2025-01-16 03:50:46,427] Trial 1 finished with value: 0.5 and parameters: {'layer_1': 77, 'layer_2': 66, 'activation': 'logistic', 'solver': 'sgd', 'learning_rate_init': 0.02875729407652898}. Best is trial 0 with value: 0.8833333333333333.
/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
[I 2025-01-16 03:50:50,483] Trial 2 finished with value: 0.8566666666666667 and parameters: {'layer_1': 56, 'layer_2': 123, 'activation': 'logistic', 'solver': 'adam', 'learning_rate_init': 0.023915772509062395}. Best is trial 0 with value: 0.8833333333333333.
[I 202

CFV dataset created and saved!


CPFV (Combined Probability and Class Feature Vector)

In [3]:
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_3/8_PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")


# Combine positive and negative datasets
main_data = pd.concat([main_p.assign(label=1), main_n.assign(label=0)], ignore_index=True)
validation_data = pd.concat([validation_p.assign(label=1), validation_n.assign(label=0)], ignore_index=True)

# Separate features and labels
X_train = main_data.drop(columns=["label"])
y_train = main_data["label"]
X_val = validation_data.drop(columns=["label"])
y_val = validation_data["label"]

# Initialize models with their tuned hyperparameters
trained_models = {
    "SVM": SVC(C=1.0, kernel="rbf", probability=True),  # Example parameters
    "Decision Tree": DecisionTreeClassifier(max_depth=10, min_samples_split=5),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=10, min_samples_split=5),
    "Logistic Regression": LogisticRegression(C=1.0, solver="lbfgs"),
    "k-NN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=10),
    "XGBoost": XGBClassifier(n_estimators=100, max_depth=10, learning_rate=0.1, use_label_encoder=False, eval_metric="logloss"),
    "LightGBM": LGBMClassifier(n_estimators=100, max_depth=10, learning_rate=0.1),
    "AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=0.1),
    "Neural Network (MLPClassifier)": MLPClassifier(hidden_layer_sizes=(100, 50), activation="relu", solver="adam", learning_rate_init=0.01, max_iter=200),
    "Multilayer Perceptron (Custom MLP)": MLPClassifier(hidden_layer_sizes=(128, 64), activation="relu", solver="adam", learning_rate_init=0.01, max_iter=200)
}

# Train all models on the training dataset
for model_name, model in trained_models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)

# Function to create CPFV dataset
def create_cpfv(models, X_data, y_data):
    cpfv_data = pd.DataFrame()
    for model_name, model in models.items():
        # Add predicted class labels
        cpfv_data[f"{model_name}_Class"] = model.predict(X_data)
        # Add predicted probabilities or decision scores
        if hasattr(model, "predict_proba"):
            cpfv_data[f"{model_name}_Prob"] = model.predict_proba(X_data)[:, 1]
        elif hasattr(model, "decision_function"):
            cpfv_data[f"{model_name}_Prob"] = model.decision_function(X_data)
        else:
            cpfv_data[f"{model_name}_Prob"] = cpfv_data[f"{model_name}_Class"]
    # Add true labels
    cpfv_data["True_Label"] = y_data.reset_index(drop=True)
    return cpfv_data

# Create CPFV dataset using validation data
cpfv_dataset = create_cpfv(trained_models, X_val, y_val)

# Save CPFV dataset to CSV
cpfv_dataset.to_csv("CPFV_PAAC.csv", index=False)

Training SVM...
Training Decision Tree...
Training Random Forest...
Training Logistic Regression...
Training k-NN...
Training Naive Bayes...
Training Gradient Boosting...
Training XGBoost...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:57:25] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000655 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 8616
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training AdaBoost...
Training Neural Network (MLPClassifier)...
Training Multilayer Perceptron (Custom MLP)...


# **Hyperperameter tune for RandomizedSearchCV**

In [ ]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier # Import path for KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

# Combine positive and negative samples
X_train = pd.concat([main_p, main_n])
y_train = np.concatenate([np.ones(len(main_p)), np.zeros(len(main_n))])

# Define cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Function to create a neural network model
def create_nn(num_units, dropout_rate, learning_rate, input_shape):
    model = Sequential([
        Dense(num_units, activation='relu', input_shape=input_shape),
        Dropout(dropout_rate),
        Dense(num_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model definitions and parameter grids for RandomizedSearchCV
models = {
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": KerasClassifier(
        model=create_nn,
        num_units=64,
        dropout_rate=0.2,
        learning_rate=0.001,
        input_shape=(X_train.shape[1],),
        epochs=5,
        batch_size=32,
        verbose=0
    )
}

# Parameter grids for each model
param_grids = {
    "SVM": {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']},
    "Decision Tree": {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    "Random Forest": {'n_estimators': [100, 200, 500], 'max_depth': [10, 20, None], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10]},
    "Logistic Regression": {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'saga'], 'penalty': ['l2']},
    "k-NN": {'n_neighbors': [3, 5, 11, 19], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
    "Naive Bayes": {'var_smoothing': np.logspace(-9, -1, 10)},
    "Gradient Boosting": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "XGBoost": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "LightGBM": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [10, 20, -1]},
    "CatBoost": {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.1, 0.2], 'iterations': [100, 200]},
    "AdaBoost": {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0]},
    "Neural Network": {
        'model__num_units': [32, 64, 128],
        'model__dropout_rate': [0.1, 0.2, 0.3],
        'model__learning_rate': [0.001, 0.01, 0.1]
    }
}

# Results storage
best_params = {}
best_scores = []

# Loop through models and apply random search
for model_name, model in models.items():
    print(f"Performing RandomizedSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform randomized search
    random_search = RandomizedSearchCV(model, param_grid, n_iter=10, cv=cv, scoring='accuracy', n_jobs=-1, random_state=42)
    random_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = random_search.best_params_
    best_scores.append(random_search.best_score_)

# Display results in a DataFrame
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Best Score': best_scores,
    'Best Parameters': [best_params[model] for model in models]
})

print(results_df)


Performing RandomizedSearchCV for SVM...
Performing RandomizedSearchCV for Decision Tree...
Performing RandomizedSearchCV for Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
20 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/utils/_p

Performing RandomizedSearchCV for Logistic Regression...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Performing RandomizedSearchCV for k-NN...
Performing RandomizedSearchCV for Naive Bayes...
Performing RandomizedSearchCV for Gradient Boosting...
Performing RandomizedSearchCV for XGBoost...
Performing RandomizedSearchCV for LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001279 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8616
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Performing RandomizedSearchCV for CatBoost...
Performing RandomizedSearchCV for AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 9 is smaller than n_iter=10. Running 9 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Performing RandomizedSearchCV for Neural Network...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


                  Model  Best Score  \
0                   SVM    0.895194   
1         Decision Tree    0.821289   
2         Random Forest    0.881427   
3   Logistic Regression    0.774893   
4                  k-NN    0.915821   
5           Naive Bayes    0.741379   
6     Gradient Boosting    0.894332   
7               XGBoost    0.880579   
8              LightGBM    0.884868   
9              CatBoost    0.889156   
10             AdaBoost    0.803241   
11       Neural Network    0.821296   

                                      Best Parameters  
0        {'kernel': 'rbf', 'gamma': 'scale', 'C': 10}  
1   {'min_samples_split': 2, 'min_samples_leaf': 2...  
2   {'n_estimators': 500, 'min_samples_split': 5, ...  
3   {'solver': 'liblinear', 'penalty': 'l2', 'C': 10}  
4   {'weights': 'distance', 'n_neighbors': 3, 'met...  
5                              {'var_smoothing': 0.1}  
6   {'n_estimators': 200, 'max_depth': 7, 'learnin...  
7   {'n_estimators': 100, 'max_depth': 5, 'l

In [ ]:
# Storage for predictions and target column
probability_datasets = pd.DataFrame(y_train, columns=['Target'])

# Loop through models, perform random search, and save probabilities
for model_name, model in models.items():
    print(f"Performing RandomizedSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform randomized search
    random_search = RandomizedSearchCV(model,
                                       param_grid,
                                       n_iter=10,
                                       cv=cv,
                                       scoring='accuracy',
                                       n_jobs=-1,
                                       random_state=42)

    random_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = random_search.best_params_
    best_scores.append(random_search.best_score_)



    # Get probability predictions (if supported)
    if hasattr(random_search.best_estimator_, "predict_proba"):
        probabilities = random_search.best_estimator_.predict_proba(X_train)[:, 1]  # Probability for the positive class
        probability_datasets[f"{model_name}_Probabilities"] = probabilities
    else:
        # Fallback if probability prediction isn't supported
        predictions = random_search.best_estimator_.predict(X_train)
        probability_datasets[f"{model_name}_Predictions"] = predictions



# Display final dataset with probabilities
print(probability_datasets.head())

# Save the probability dataset to a CSV file
probability_datasets.to_csv("PAAC_Randomsearch_tune.csv", index=False)
print("Probability dataset saved to 'PAAC_Randomsearch_tune.csv'.")


# **Hyperparameter grids for GridSearchCV**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_main_negative.csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_validation_positive.csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/PAAC (Pseudo Amino Acid Composition)/paac_validation_negative.csv")

# Combine positive and negative samples
X_train = pd.concat([main_p, main_n])
y_train = np.concatenate([np.ones(len(main_p)), np.zeros(len(main_n))])

# Define cross-validation
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Function to create a neural network model
def create_nn(num_units, dropout_rate, learning_rate, input_shape):
    model = Sequential([
        Dense(num_units, activation='relu', input_shape=input_shape),
        Dropout(dropout_rate),
        Dense(num_units, activation='relu'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Model definitions and parameter grids for GridSearchCV
models = {
    "SVM": SVC(),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "Logistic Regression": LogisticRegression(),
    "k-NN": KNeighborsClassifier(),
    "Naive Bayes": GaussianNB(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "XGBoost": XGBClassifier(),
    "LightGBM": LGBMClassifier(),
    "CatBoost": CatBoostClassifier(verbose=0),
    "AdaBoost": AdaBoostClassifier(),
    "Neural Network": KerasClassifier(
        model=create_nn,
        num_units=64,
        dropout_rate=0.2,
        learning_rate=0.001,
        input_shape=(X_train.shape[1],),
        epochs=5,
        batch_size=32,
        verbose=0
    )
}

# Parameter grids for each model
param_grids = {
    "SVM": {'C': [0.1, 1, 10, 100], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']},
    "Decision Tree": {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4]},
    "Random Forest": {'n_estimators': [100, 200, 500], 'max_depth': [10, 20, None], 'max_features': ['auto', 'sqrt'], 'min_samples_split': [2, 5, 10]},
    "Logistic Regression": {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'saga'], 'penalty': ['l2']},
    "k-NN": {'n_neighbors': [3, 5, 11, 19], 'weights': ['uniform', 'distance'], 'metric': ['euclidean', 'manhattan']},
    "Naive Bayes": {'var_smoothing': np.logspace(-9, -1, 10)},
    "Gradient Boosting": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "XGBoost": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [3, 5, 7]},
    "LightGBM": {'n_estimators': [100, 200], 'learning_rate': [0.01, 0.1, 0.2], 'max_depth': [10, 20, -1]},
    "CatBoost": {'depth': [6, 8, 10], 'learning_rate': [0.01, 0.1, 0.2], 'iterations': [100, 200]},
    "AdaBoost": {'n_estimators': [50, 100, 200], 'learning_rate': [0.01, 0.1, 1.0]},
    "Neural Network": {
        'model__num_units': [32, 64, 128],
        'model__dropout_rate': [0.1, 0.2, 0.3],
        'model__learning_rate': [0.001, 0.01, 0.1]
    }
}

# Results storage
best_params = {}
best_scores = []

# Loop through models and apply grid search
for model_name, model in models.items():
    print(f"Performing GridSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform grid search
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = grid_search.best_params_
    best_scores.append(grid_search.best_score_)

# Display results in a DataFrame
results_df = pd.DataFrame({
    'Model': list(models.keys()),
    'Best Score': best_scores,
    'Best Parameters': [best_params[model] for model in models]
})

print(results_df)


Performing GridSearchCV for SVM...
Performing GridSearchCV for Decision Tree...
Performing GridSearchCV for Random Forest...


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
135 fits failed out of a total of 270.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1466, in wrapper
    estimator._validate_params()
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/util

Performing GridSearchCV for Logistic Regression...
Performing GridSearchCV for k-NN...


/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Performing GridSearchCV for Naive Bayes...
Performing GridSearchCV for Gradient Boosting...
Performing GridSearchCV for XGBoost...
Performing GridSearchCV for LightGBM...


...

[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001166 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 8616
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 50
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Performing GridSearchCV for CatBoost...
Performing GridSearchCV for AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Performing GridSearchCV for Neural Network...


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


                  Model  Best Score  \
0                   SVM    0.899497   
1         Decision Tree    0.825596   
2         Random Forest    0.888297   
3   Logistic Regression    0.774893   
4                  k-NN    0.915821   
5           Naive Bayes    0.741379   
6     Gradient Boosting    0.894321   
7               XGBoost    0.882289   
8              LightGBM    0.885726   
9              CatBoost    0.900326   
10             AdaBoost    0.803241   
11       Neural Network    0.847073   

                                      Best Parameters  
0       {'C': 100, 'gamma': 'scale', 'kernel': 'rbf'}  
1   {'max_depth': 10, 'min_samples_leaf': 2, 'min_...  
2   {'max_depth': 20, 'max_features': 'sqrt', 'min...  
3   {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}  
4   {'metric': 'manhattan', 'n_neighbors': 3, 'wei...  
5                              {'var_smoothing': 0.1}  
6   {'learning_rate': 0.2, 'max_depth': 7, 'n_esti...  
7   {'learning_rate': 0.1, 'max_depth': 7, '

In [ ]:
# Prepare the final dataset with probabilities and target
all_probabilities = []
all_targets = []

# Loop through models and apply grid search
for model_name, model in models.items():
    print(f"Performing GridSearchCV for {model_name}...")
    param_grid = param_grids[model_name]

    # Perform grid search
    grid_search = GridSearchCV(model, param_grid, cv=cv, scoring='accuracy', n_jobs=-1)
    grid_search.fit(X_train, y_train)

    # Store best parameters and score
    best_params[model_name] = grid_search.best_params_
    best_scores.append(grid_search.best_score_)

    # Predict probabilities using the best estimator
    best_model = grid_search.best_estimator_
    probabilities = best_model.predict_proba(X_train)[:, 1]  # Positive class probabilities

    # Append probabilities and targets for this model
    all_probabilities.append(probabilities)
    all_targets.append(y_train)

    # Combine probabilities, features, and target into a DataFrame
    model_data = X_train.copy()
    model_data[f"{model_name}_probability"] = probabilities
    model_data['target'] = y_train

    # Save to CSV
    output_path = f"/content/{model_name}_probabilities.csv"
    model_data.to_csv(output_path, index=False)
    print(f"Saved probabilities for {model_name} to {output_path}")

# Combine all model probabilities into a single DataFrame (optional)
final_dataset = pd.DataFrame({'target': y_train})
for idx, model_name in enumerate(models.keys()):
    final_dataset[f"{model_name}_probability"] = all_probabilities[idx]

# Save the combined dataset
final_output_path = "/content/combined_probabilities.csv"
final_dataset.to_csv(final_output_path, index=False)
print(f"Saved combined dataset to {final_output_path}")
